# Import and Tidy Data

In this notebook we will import the European Soccer dataset from the sqlite file, query the tables and columns that we need and clean the data. 

## Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from seaborn import load_dataset
from sqlalchemy import create_engine
from pandas.io import sql
from sqlalchemy import inspect
import pyarrow.parquet as pq
import pyarrow as pa

 ### Connect to SQL file

In [2]:
sc = create_engine('sqlite:////data/bnguy123/datasets/hugomathien/soccer/database.sqlite')

### Inspect the data

First, I need to inspect the data in order to decide which table and columns should I query to work on

In [3]:
inspector = inspect(sc)
inspector.get_table_names()

['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes',
 'sqlite_sequence']

In [4]:
def table_info(table_name, i):
    """Show the column names and types as a Pandas DataFrame."""
    return pd.DataFrame(i.get_columns(table_name))[['name','type']]

In [5]:
table_info('Team', inspector)

,name,type
0,id,INTEGER
1,team_api_id,INTEGER
2,team_fifa_api_id,INTEGER
3,team_long_name,TEXT
4,team_short_name,TEXT


In [6]:
table_info('Match', inspector)

,name,type
0,id,INTEGER
1,country_id,INTEGER
2,league_id,INTEGER
3,season,TEXT
4,stage,INTEGER
5,date,TEXT
6,match_api_id,INTEGER
7,home_team_api_id,INTEGER
8,away_team_api_id,INTEGER
9,home_team_goal,INTEGER


In [7]:
table_info('Team_Attributes', inspector)

,name,type
0,id,INTEGER
1,team_fifa_api_id,INTEGER
2,team_api_id,INTEGER
3,date,TEXT
4,buildUpPlaySpeed,INTEGER
5,buildUpPlaySpeedClass,TEXT
6,buildUpPlayDribbling,INTEGER
7,buildUpPlayDribblingClass,TEXT
8,buildUpPlayPassing,INTEGER
9,buildUpPlayPassingClass,TEXT


In [8]:
table_info('country', inspector)

,name,type
0,id,INTEGER
1,name,TEXT


### Query the tables and columns we need

To work with this dataset, I just only focus on the attribute of all soccer team over nine countries and the result of the soccer game from season 2006/2007. So I only query the columns from 4 table team, team attribute, match and country

In [9]:
team = sql.read_sql_query("""SELECT team_api_id, team_short_name FROM Team""", sc)
team_att = sql.read_sql_query("""SELECT team_api_id, date, buildUpPlaySpeed, buildUpPlayDribbling,
buildUpPlayPassing, chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, 
defencePressure, defenceAggression, defenceTeamWidth FROM Team_Attributes""", sc)
country = sql.read_sql_table('Country',sc)
match = sql.read_sql_query("""SELECT match_api_id, country_id, season, home_team_api_id,
away_team_api_id, home_team_goal, away_team_goal FROM Match""", sc)

In [10]:
match.head()

,match_api_id,country_id,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,492473,1,2008/2009,9987,9993,1,1
1,492474,1,2008/2009,10000,9994,0,0
2,492475,1,2008/2009,9984,8635,0,3
3,492476,1,2008/2009,9991,9998,5,0
4,492477,1,2008/2009,7947,9985,1,3


## Transform and Tidy the data

First look at the team attribute dataframe

In [11]:
team_att.head()

,team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,9930,2010-02-22 00:00:00,60,NaN,50,60,65,55,50,55,45
1,9930,2014-09-19 00:00:00,52,48.0,56,54,63,64,47,44,54
2,9930,2015-09-10 00:00:00,47,41.0,54,54,63,64,47,44,54
3,8485,2010-02-22 00:00:00,70,NaN,70,70,70,70,60,70,70
4,8485,2011-02-22 00:00:00,47,NaN,52,53,48,52,47,47,52


In [12]:
team_att.date.unique()

array(['2010-02-22 00:00:00', '2014-09-19 00:00:00',
       '2015-09-10 00:00:00', '2011-02-22 00:00:00',
       '2012-02-22 00:00:00', '2013-09-20 00:00:00'], dtype=object)

This dataframe contain the new as well as the old attributes of the team, while we just only want to work with the newest attribute so I do the filter.

In [13]:
team_att = team_att[team_att['date'] == '2015-09-10 00:00:00']

In [14]:
team_att.head()

,team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
2,9930,2015-09-10 00:00:00,47,41.0,54,54,63,64,47,44,54
8,8485,2015-09-10 00:00:00,59,64.0,53,51,72,63,49,45,63
14,8576,2015-09-10 00:00:00,59,57.0,52,48,38,52,38,47,53
20,8564,2015-09-10 00:00:00,48,70.0,52,66,36,69,58,57,49
26,10215,2015-09-10 00:00:00,53,53.0,44,55,51,60,39,38,61


I also want to add the name of country of the team to the data frame, so I merge the two dataframe 

In [15]:
team_country = match[['home_team_api_id', 'country_id']]
team_country = team_country.drop_duplicates()
team_att = pd.merge(team_att,team_country, left_on ='team_api_id' , right_on='home_team_api_id')
team_att = pd.merge(team_att,country, left_on='country_id', right_on='id')

In [16]:
team_att.head()

,team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth,home_team_api_id,country_id,id,name
0,9930,2015-09-10 00:00:00,47,41.0,54,54,63,64,47,44,54,9930,24558,24558,Switzerland
1,9931,2015-09-10 00:00:00,61,63.0,46,66,66,54,47,58,53,9931,24558,24558,Switzerland
2,9824,2015-09-10 00:00:00,53,32.0,56,38,53,46,42,33,58,9824,24558,24558,Switzerland
3,10243,2015-09-10 00:00:00,62,49.0,46,47,50,54,47,43,56,10243,24558,24558,Switzerland
4,9956,2015-09-10 00:00:00,50,60.0,54,52,54,46,36,43,47,9956,24558,24558,Switzerland


Change the column names and gether the columns have the similar information

In [17]:
team_column_names = {
    'team_api_id' : 'team_id','date':'date_update', 'buildUpPlaySpeed' : 'speed', 
    'buildUpPlayDribbling': 'dribbling', 'passing':'passing','chanceCreationCrossing' : 
    'crossing','chanceCreationShooting': 'shooting','defence':'defence','name':'country_name' }
team_att['passing'] = (team_att['buildUpPlayPassing'] + team_att['chanceCreationPassing'])//2
team_att['defence'] = (team_att['defencePressure'] + team_att['defenceAggression'] + team_att['defenceTeamWidth'])//3
team_att = team_att[[
    'team_api_id','date','buildUpPlaySpeed', 'buildUpPlayDribbling','passing',
    'chanceCreationCrossing','chanceCreationShooting','defence','name' ]]
team_att = (team_att.rename(columns=team_column_names).set_index('team_id').sort_index())

In [18]:
team_att.head()

,date_update,speed,dribbling,passing,crossing,shooting,defence,country_name
team_id,,,,,,,,
1601,2015-09-10 00:00:00,47,48.0,49,65,50,45,Poland
1957,2015-09-10 00:00:00,66,54.0,45,33,66,54,Poland
2182,2015-09-10 00:00:00,57,50.0,49,70,67,51,Poland
2186,2015-09-10 00:00:00,37,50.0,52,66,33,52,Poland
4087,2015-09-10 00:00:00,58,35.0,54,61,51,46,France


Make some assert tests

In [19]:
assert team_att.shape == (245, 8)
assert list(team_att.columns)==['date_update','speed', 'dribbling','passing',
                                'crossing','shooting','defence','country_name']
assert len(team_att.country_name.unique()) == 11
assert len(team_att.date_update.unique()) == 1
for i in country.name:
    assert i in team_att.country_name.unique() 


Check if there is any null value in our dataframe

In [20]:
team_att.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 1601 to 274581
Data columns (total 8 columns):
date_update     245 non-null object
speed           245 non-null int64
dribbling       245 non-null float64
passing         245 non-null int64
crossing        245 non-null int64
shooting        245 non-null int64
defence         245 non-null int64
country_name    245 non-null object
dtypes: float64(1), int64(5), object(2)
memory usage: 17.2+ KB


In [21]:
team_att.head()

,date_update,speed,dribbling,passing,crossing,shooting,defence,country_name
team_id,,,,,,,,
1601,2015-09-10 00:00:00,47,48.0,49,65,50,45,Poland
1957,2015-09-10 00:00:00,66,54.0,45,33,66,54,Poland
2182,2015-09-10 00:00:00,57,50.0,49,70,67,51,Poland
2186,2015-09-10 00:00:00,37,50.0,52,66,33,52,Poland
4087,2015-09-10 00:00:00,58,35.0,54,61,51,46,France


There is no null values in team attribute data frame so now we good with this dataframe, we move to the second one, match dataframe

In [22]:
match.head()

,match_api_id,country_id,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,492473,1,2008/2009,9987,9993,1,1
1,492474,1,2008/2009,10000,9994,0,0
2,492475,1,2008/2009,9984,8635,0,3
3,492476,1,2008/2009,9991,9998,5,0
4,492477,1,2008/2009,7947,9985,1,3


Since this table only have the home and away team goal, we want to add a column of result of home team, and also the team short_name for these both team besides the team id

In [23]:
match = pd.merge(match, team, left_on='home_team_api_id', right_on='team_api_id').drop('team_api_id',axis=1)
match = pd.merge(match, team, left_on='away_team_api_id', right_on='team_api_id').drop('team_api_id',axis=1)
match = pd.merge(match, country, left_on = 'country_id', right_on = 'id').drop(['country_id','id'],axis=1)
match = match.rename(index=str, columns={"team_short_name_x": "home_team", \
                                         "team_short_name_y": "away_team","name":"country_name"})


In [24]:
match.set_index('match_api_id', inplace=True)
match.head()

,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team,away_team,country_name
match_api_id,,,,,,,,
492473,2008/2009,9987,9993,1,1,GEN,BAC,Belgium
665353,2009/2010,9987,9993,1,1,GEN,BAC,Belgium
838476,2010/2011,9987,9993,2,1,GEN,BAC,Belgium
1032694,2011/2012,9987,9993,3,1,GEN,BAC,Belgium
1224064,2012/2013,9987,9993,3,0,GEN,BAC,Belgium


In [25]:
def get_result(x):
    """This function will convert the score between 2 team to the result of the game"""
    if(x<0):
        return 'loose'
    elif(x==0):
        return 'tie'
    else:
        return 'win'

match['result'] = match['home_team_goal']-match['away_team_goal']
match['result'] = match['result'].apply(get_result)

In [26]:
match.head()

,season,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_team,away_team,country_name,result
match_api_id,,,,,,,,,
492473,2008/2009,9987,9993,1,1,GEN,BAC,Belgium,tie
665353,2009/2010,9987,9993,1,1,GEN,BAC,Belgium,tie
838476,2010/2011,9987,9993,2,1,GEN,BAC,Belgium,win
1032694,2011/2012,9987,9993,3,1,GEN,BAC,Belgium,win
1224064,2012/2013,9987,9993,3,0,GEN,BAC,Belgium,win


In [27]:
assert match.shape == (25979,9)
assert len(match.home_team.unique()) == len(team.team_short_name.unique())
assert len(match.away_team.unique()) == len(team.team_short_name.unique())
assert len(match.result.unique()) == 3
assert 'tie' in match.result.unique()
assert 'win' in match.result.unique()
assert 'loose' in match.result.unique()
assert len(match[match['result'] == 'win']) == \
len(match[(match['home_team_goal']-match['away_team_goal']) > 0])
assert len(match[match['result'] == 'loose']) == \
len(match[(match['home_team_goal']-match['away_team_goal']) < 0])
assert len(match[match['result'] == 'tie']) == \
len(match[(match['home_team_goal']-match['away_team_goal']) == 0])

Also check if this dataframe has any null values

In [28]:
match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25979 entries, 492473 to 1992206
Data columns (total 9 columns):
season              25979 non-null object
home_team_api_id    25979 non-null int64
away_team_api_id    25979 non-null int64
home_team_goal      25979 non-null int64
away_team_goal      25979 non-null int64
home_team           25979 non-null object
away_team           25979 non-null object
country_name        25979 non-null object
result              25979 non-null object
dtypes: int64(4), object(5)
memory usage: 2.0+ MB


This dataframe is look good, now we need to save it. 

## Write DataFrame to a parquet file

In [29]:
filename = '/data/ellisonbg/cars.parquet'
team_table = pa.Table.from_pandas(team_att)
match_table = pa.Table.from_pandas(match)
pq.write_table(team_table, '/data/bnguy123/team_table.parquet')
pq.write_table(match_table, '/data/bnguy123/match_table.parquet')


Ok, let move to second part. Make some EDA to answers some of the questions about the dataset

# EDA

* [EDA](03-EDA.ipynb)